# Bayesian RCT Lean Modeling

## Introduction

This section implements the **Bayesian RCT Lean modeling approach** for predicting control arm outcomes with uncertainty quantification. Unlike the point estimate approach in the main notebook, this provides full posterior distributions for:

- **Control arm predictions**: Complete uncertainty around predicted outcomes
- **ATE estimates**: Probabilistic treatment effect predictions
- **Clinical decision making**: Risk-based trial go/no-go decisions

### Key Advantages of Bayesian Approach:
- **Uncertainty quantification**: Know confidence in predictions
- **Risk assessment**: Calculate probability of positive/negative effects
- **Sequential learning**: Update beliefs as new trial data arrives
- **Clinical interpretation**: Natural probabilistic language for clinicians

## Load RCT Data and Configuration

In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import json
import os
%load_ext autoreload

In [2]:
# Import utility functions
from utilities.utils import *

In [3]:
# Load RCT training data
training_df = pd.read_csv("processed_training_data.csv")
training_df.head()

,NCT_ID,Arm,Population,intervention,RCT_with_control_inter,gender_male_percent,age_median,no_smoker_percent,ecog_1,brain_metastase_yes,...,is_arm_control,combo_therapy,egfr_targeted,egfr_tki_use,high_risk_profile,novelty_score,elderly_male,large_trial,treatment_complexity,smoker_percent
0,NCT01364012,Intervention,138.0,bevacizumab + platinum doublet chemo (carbopla...,1.0,54.0,57.0,50.0,75.0,0.0,...,0,1,0.0,0.0,1,1.0,0,0,3.0,50.0
1,NCT01364012,Control,138.0,placebo + platinum doublet chemo (carboplatin ...,1.0,56.0,56.0,50.0,80.0,0.0,...,1,0,0.0,0.0,1,0.0,0,0,1.0,50.0
2,NCT01469000,Intervention,126.0,pemetrexed + gefitinib,1.0,35.0,62.0,64.0,69.0,0.0,...,0,1,100.0,100.0,1,1.0,0,0,2.0,36.0
3,NCT01469000,Control,65.0,gefitinib,1.0,37.0,62.0,72.0,68.0,0.0,...,1,0,100.0,100.0,1,1.0,0,0,1.0,28.0
4,NCT02099058,Intervention,28.0,telisotuzumab vedotin + erlotinib,0.0,32.0,60.0,0.0,71.0,0.0,...,0,0,100.0,100.0,1,1.0,0,0,2.0,100.0


In [4]:
# Load feature configuration
config_path = os.path.join('config', 'feature_config.json')
with open(config_path, 'r') as f:
    feature_config = json.load(f)


# Extract configuration
target_col = feature_config.get('target', 'PFS_median_months')
trial_id_col = feature_config.get('trial_id_column', 'NCT_ID')
control_arm_col = feature_config.get('control_arm_column', 'is_arm_control')
intervention_outcome_col = feature_config.get('intervention_outcome_column', 'intervention_outcome')


In [5]:
# Use subset of most important features for Bayesian modeling
bayesian_features = [
    'age_median', 'gender_male_percent', 'ecog_1', 
    'EGFR_positive_mutation', 'disease_stage_IV',
    'PD1_PDL1_Inhibitor', 'EGFR_TKI', 'Platinum_Chemotherapy'
]

print(f"Dataset shape: {training_df.shape}")
print(f"Number of trials: {training_df[trial_id_col].nunique()}")
print(f"Bayesian features: {bayesian_features}")

Dataset shape: (57, 45)
Number of trials: 27
Bayesian features: ['age_median', 'gender_male_percent', 'ecog_1', 'EGFR_positive_mutation', 'disease_stage_IV', 'PD1_PDL1_Inhibitor', 'EGFR_TKI', 'Platinum_Chemotherapy']


## Bayesian Control Arm Prediction Model

### Model Specification:

We build a hierarchical Bayesian model for control arm outcomes:

```
PFS_control ~ Normal(μ, σ)
μ = α + Σ(βᵢ × featureᵢ) + β_intervention × PFS_intervention
```

**Priors:**
- Weakly informative priors based on clinical knowledge
- Regularization through prior specification
- **No standardization**: Mixed feature types (continuous + one-hot encoded)
- **Missing value handling**: Using existing prepare_data and run_fillna functions

In [6]:
def prepare_bayesian_data(df, features, target_col, trial_id_col, control_arm_col):
    """Prepare data for Bayesian modeling with proper missing value handling"""
    
    # Start with control arms only
    control_data = df[df[control_arm_col] == 1].copy()
    
    # Add intervention outcomes for each control arm
    control_data[intervention_outcome_col] = np.nan
    
    for idx, row in control_data.iterrows():
        trial_id = row[trial_id_col]
        # Find intervention arm for this trial
        intervention_data = df[(df[trial_id_col] == trial_id) & (df[control_arm_col] != 1)]
        if not intervention_data.empty:
            control_data.loc[idx, intervention_outcome_col] = intervention_data[target_col].mean()
    
    # Remove rows without intervention outcomes
    control_data = control_data.dropna(subset=[intervention_outcome_col])
    
    # Use existing data preparation functions instead of manual standardization
    print(f"Raw data shape: {control_data.shape}")
    print(f"Features to use: {features}")
    
    # Prepare features using existing function (handles missing values properly)
    feature_cols = features + [intervention_outcome_col, target_col]
    prepared_data = prepare_data(control_data, feature_cols)
    prepared_data = run_fillna(prepared_data)
    
    # Add back the trial ID for tracking
    prepared_data[trial_id_col] = control_data[trial_id_col].values
    
    print(f"Prepared data shape: {prepared_data.shape}")
    print(f"Missing values after preparation: {prepared_data.isnull().sum().sum()}")
    
    return prepared_data

# Prepare data using existing preprocessing pipeline
bayesian_data = prepare_bayesian_data(
    training_df, bayesian_features, target_col, trial_id_col, control_arm_col
)

print(f"\nFinal Bayesian modeling dataset:")
print(f"Shape: {bayesian_data.shape}")
print(f"Columns: {list(bayesian_data.columns)}")
print(f"Target variable stats: mean={bayesian_data[target_col].mean():.2f}, std={bayesian_data[target_col].std():.2f}")

Raw data shape: (24, 46)
Features to use: ['age_median', 'gender_male_percent', 'ecog_1', 'EGFR_positive_mutation', 'disease_stage_IV', 'PD1_PDL1_Inhibitor', 'EGFR_TKI', 'Platinum_Chemotherapy']
Prepared data shape: (24, 11)
Missing values after preparation: 0

Final Bayesian modeling dataset:
Shape: (24, 11)
Columns: ['age_median', 'gender_male_percent', 'ecog_1', 'EGFR_positive_mutation', 'disease_stage_IV', 'PD1_PDL1_Inhibitor', 'EGFR_TKI', 'Platinum_Chemotherapy', 'intervention_outcome', 'PFS_median_months', 'NCT_ID']
Target variable stats: mean=6.32, std=1.77


In [7]:
bayesian_features

['age_median',
 'gender_male_percent',
 'ecog_1',
 'EGFR_positive_mutation',
 'disease_stage_IV',
 'PD1_PDL1_Inhibitor',
 'EGFR_TKI',
 'Platinum_Chemotherapy']

In [8]:
bayesian_data

,age_median,gender_male_percent,ecog_1,EGFR_positive_mutation,disease_stage_IV,PD1_PDL1_Inhibitor,EGFR_TKI,Platinum_Chemotherapy,intervention_outcome,PFS_median_months,NCT_ID
1,56.000000,56.00,80.000,26.000000,91.000000,0.0,0.0,1.0,9.20,6.5,NCT01364012
3,62.000000,37.00,68.000,100.000000,88.000000,0.0,1.0,0.0,15.80,10.9,NCT01469000
6,64.000000,39.00,62.800,100.000000,100.000000,0.0,0.0,1.0,5.60,5.5,NCT03515837
9,62.000000,34.00,58.000,100.000000,95.000000,0.0,0.0,1.0,9.60,9.6,NCT04129502
11,61.000000,59.50,79.700,54.000000,100.000000,0.0,0.0,1.0,9.50,7.1,NCT04194203
13,62.000000,40.00,65.000,100.000000,100.000000,0.0,0.0,1.0,11.40,6.7,NCT04538664
16,62.000000,40.00,62.000,100.000000,100.000000,0.0,0.0,1.0,7.30,4.2,NCT04988295
20,66.000000,62.90,64.900,0.000000,100.000000,0.0,0.0,1.0,7.70,5.5,NCT02142738
22,65.000000,55.00,64.000,0.000000,90.000000,0.0,0.0,1.0,4.20,5.9,NCT02041533
24,63.000000,67.10,59.900,1.700000,100.000000,0.0,0.0,1.0,7.60,5.2,NCT02657434


In [9]:
def build_bayesian_control_model(data, features, target_col, intervention_outcome_col):
    """Build Bayesian model with proper handling for mixed feature types and limited data"""
    
    print(f"Building model with {len(features)} features and {len(data)} observations")
    
    with pm.Model() as model:
        # More conservative priors for limited data
        intercept = pm.Normal('intercept', mu=6.0, sigma=3.0)  # Based on typical PFS values
        
        # Feature coefficients with conservative priors
        beta_features = {}
        for feature in features:
            if feature in data.columns:
                # Smaller prior variance for limited data to avoid overfitting
                beta_features[feature] = pm.Normal(f'beta_{feature}', mu=0, sigma=1.0)
        
        # Intervention outcome coefficient (key for RCT Lean)
        # Positive prior but conservative
        beta_intervention = pm.Normal('beta_intervention', mu=0.3, sigma=0.5)
        
        # More conservative noise prior for limited data
        sigma = pm.HalfNormal('sigma', sigma=2.0)
        
        # Linear combination - no standardization needed
        mu = intercept + beta_intervention * data[intervention_outcome_col]
        
        for feature in features:
            if feature in data.columns:
                mu += beta_features[feature] * data[feature]
        
        # Likelihood
        y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=data[target_col])
        
        # More conservative sampling for limited data
        trace = pm.sample(
            draws=1500,        # Fewer draws
            tune=750,          # Fewer tuning steps
            # chains=2,          # Fewer chains
            target_accept=0.9, # Higher target accept for stability
            random_seed=42, 
            return_inferencedata=True,
            progressbar=True,
        )

        print("Computing log-likelihood...")
        trace = pm.compute_log_likelihood(trace)
        # Calculate model comparison metrics
        print("\nCalculating model assessment metrics...")
        # WAIC (Widely Applicable Information Criterion)
        waic_data = az.waic(trace)
        print(f"WAIC: {waic_data.elpd_waic:.2f} ± {waic_data.se:.2f}")
        # LOO (Leave-One-Out Cross-Validation)
        loo_data = az.loo(trace)
        print(f"LOO: {loo_data.elpd_loo:.2f} ± {loo_data.se:.2f}")
        # Print additional diagnostics
        print(f"WAIC effective number of parameters: {waic_data.p_waic:.2f}")
        print(f"LOO effective number of parameters: {loo_data.p_loo:.2f}")
        pre_waic, pre_loo = waic_data.elpd_waic, loo_data.elpd_loo
    
    return model, trace

# Build and fit the updated model
print("\nBuilding Bayesian control arm prediction model...")
bayesian_model, bayesian_trace = build_bayesian_control_model(
    bayesian_data, bayesian_features, target_col, intervention_outcome_col
)

print("\nModel summary:")
summary_df = az.summary(bayesian_trace, hdi_prob=0.95)
summary_df


Building Bayesian control arm prediction model...
Building model with 8 features and 24 observations


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_age_median, beta_gender_male_percent, beta_ecog_1, beta_EGFR_positive_mutation, beta_disease_stage_IV, beta_PD1_PDL1_Inhibitor, beta_EGFR_TKI, beta_Platinum_Chemotherapy, beta_intervention, sigma]


Output()

Sampling 4 chains for 750 tune and 1_500 draw iterations (3_000 + 6_000 draws total) took 8 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


Computing log-likelihood...


Output()


Calculating model assessment metrics...
WAIC: -39.64 ± 2.65
LOO: -40.30 ± 2.76
WAIC effective number of parameters: 5.80
LOO effective number of parameters: 6.45

Model summary:


/opt/homebrew/Caskroom/miniforge/base/envs/rct_opt_lean/lib/python3.12/site-packages/arviz/stats/stats.py:1667: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/rct_opt_lean/lib/python3.12/site-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,5.945,3.014,-0.046,11.810,0.042,0.041,5054.0,4208.0,1.0
beta_age_median,0.087,0.084,-0.070,0.259,0.002,0.001,2121.0,2881.0,1.0
beta_gender_male_percent,-0.060,0.030,-0.119,0.000,0.001,0.000,2195.0,2992.0,1.0
beta_ecog_1,0.015,0.031,-0.048,0.075,0.001,0.000,3467.0,3836.0,1.0
beta_EGFR_positive_mutation,-0.008,0.009,-0.025,0.010,0.000,0.000,4070.0,3683.0,1.0
beta_disease_stage_IV,-0.073,0.041,-0.159,0.004,0.001,0.001,2522.0,3343.0,1.0
beta_PD1_PDL1_Inhibitor,0.333,0.672,-0.927,1.668,0.009,0.009,5567.0,4005.0,1.0
beta_EGFR_TKI,0.127,0.870,-1.598,1.792,0.012,0.011,5105.0,3961.0,1.0
beta_Platinum_Chemotherapy,0.561,0.630,-0.734,1.740,0.008,0.008,5713.0,4498.0,1.0
beta_intervention,0.491,0.100,0.299,0.690,0.001,0.001,5350.0,4085.0,1.0


# Find priors from dataset

# Cheatsheet depending on which prior to choose for your dataset

![Prior Table](../images/prior_bayesian.jpeg)

In [10]:
def analyze_features_and_priors(data, features, target_col):
    """
    Analyze feature scales and their relationship to target to set appropriate priors
    """
    print("FEATURE ANALYSIS FOR PRIOR SELECTION:")
    print("=" * 60)
    
    feature_stats = {}
    
    for feature in features:
        if feature in data.columns:
            values = data[feature]
            feature_stats[feature] = {
                'mean': values.mean(),
                'std': values.std(),
                'min': values.min(),
                'max': values.max(),
                'range': values.max() - values.min()
            }
    
    # Display feature statistics
    stats_df = pd.DataFrame(feature_stats).T
    print("Feature Statistics:")
    print(stats_df.round(2))
    
    # Check correlation with target
    correlations = {}
    for feature in features:
        if feature in data.columns:
            corr = data[feature].corr(data[target_col])
            correlations[feature] = corr
    
    corr_df = pd.DataFrame(list(correlations.items()), columns=['Feature', 'Correlation'])
    corr_df = corr_df.sort_values('Correlation', key=abs, ascending=False)
    
    print(f"\nCorrelations with {target_col}:")
    print(corr_df.round(3))
    
    # PRIOR SELECTION REASONING
    print(f"\nPRIOR SELECTION ANALYSIS:")
    print("=" * 40)
    
    target_std = data[target_col].std()
    
    print(f"Target variable std: {target_std:.2f}")
    print(f"If a feature changes by 1 unit, how much should target change?")
    
    # Calculate effect sizes
    print(f"\nExpected effect sizes (rule of thumb):")
    print(f"- Small effect: ±{0.2 * target_std:.2f} months")
    print(f"- Medium effect: ±{0.5 * target_std:.2f} months") 
    print(f"- Large effect: ±{0.8 * target_std:.2f} months")
    
    # Check if standardization is needed
    print(f"\nSTANDARDIZATION ANALYSIS:")
    print("=" * 30)
    
    need_standardization = False
    for feature in features:
        if feature in data.columns:
            feature_range = stats_df.loc[feature, 'range']
            if feature_range > 10:  # Arbitrary threshold
                print(f"⚠️  {feature}: range = {feature_range:.1f} (large scale)")
                need_standardization = True
            else:
                print(f"✓ {feature}: range = {feature_range:.1f} (reasonable scale)")
    
    if need_standardization:
        print(f"\n🚨 RECOMMENDATION: Standardize features before modeling!")
        print(f"Reason: Large-scale features will dominate with current priors")
    else:
        print(f"\n✓ Feature scales are reasonable for current priors")
    
    return stats_df, corr_df, need_standardization

# Analyze your features
feature_stats, correlations, needs_std = analyze_features_and_priors(
    bayesian_data, bayesian_features, target_col
)

FEATURE ANALYSIS FOR PRIOR SELECTION:
Feature Statistics:
                         mean    std   min    max  range
age_median              62.89   2.16  56.0   66.0   10.0
gender_male_percent     61.57  15.80  34.0   92.1   58.1
ecog_1                  68.94   8.60  53.5   87.6   34.1
EGFR_positive_mutation  49.04  33.13   0.0  100.0  100.0
disease_stage_IV        91.37   8.61  63.6  100.0   36.4
PD1_PDL1_Inhibitor       0.08   0.28   0.0    1.0    1.0
EGFR_TKI                 0.04   0.20   0.0    1.0    1.0
Platinum_Chemotherapy    0.88   0.34   0.0    1.0    1.0

Correlations with PFS_median_months:
                  Feature  Correlation
6                EGFR_TKI        0.552
1     gender_male_percent       -0.374
3  EGFR_positive_mutation        0.272
0              age_median       -0.242
7   Platinum_Chemotherapy       -0.142
5      PD1_PDL1_Inhibitor        0.093
2                  ecog_1       -0.035
4        disease_stage_IV       -0.013

PRIOR SELECTION ANALYSIS:
Target variab

## Recommended Prior Distribution Strategy

Based on our data analysis and clinical domain knowledge:

### 1. **Normal Priors** - Our Current Choice ✅
- **Continuous outcomes**: target outcome is continuous, centered around specific values. except outliers
- **Limited data**: Normal priors provide regularization without being too restrictive


In [ ]:
# Add this cell to analyze your target variable and justify intercept prior
def analyze_target_and_set_priors(data, target_col):
    """
    Analyze target variable to set informed priors for intercept and coefficients
    """
    target_values = data[target_col]
    
    print("TARGET VARIABLE ANALYSIS:")
    print("=" * 50)
    print(f"Target: {target_col}")
    print(f"Mean: {target_values.mean():.2f}")
    print(f"Median: {target_values.median():.2f}")
    print(f"Std: {target_values.std():.2f}")
    print(f"Min: {target_values.min():.2f}")
    print(f"Max: {target_values.max():.2f}")
    print(f"25th percentile: {target_values.quantile(0.25):.2f}")
    print(f"75th percentile: {target_values.quantile(0.75):.2f}")
    
    # Plot distribution
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Histogram
    axes[0].hist(target_values, bins=20, alpha=0.7, density=True)
    axes[0].axvline(target_values.mean(), color='red', linestyle='--', label=f'Mean: {target_values.mean():.2f}')
    axes[0].axvline(target_values.median(), color='orange', linestyle='--', label=f'Median: {target_values.median():.2f}')
    axes[0].set_xlabel(target_col)
    axes[0].set_ylabel('Density')
    axes[0].set_title('Target Variable Distribution')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Q-Q plot to check normality
    from scipy import stats as scipy_stats
    scipy_stats.probplot(target_values, dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot (Normal Distribution)')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # INTERCEPT PRIOR REASONING
    print("\nINTERCEPT PRIOR REASONING:")
    print("=" * 50)
    
    # When all features are at their mean values (approximately 0 after standardization),
    # what should we expect the target to be?
    target_mean = target_values.mean()
    target_std = target_values.std()
    
    print(f"When all features are at average levels:")
    print(f"Expected outcome ≈ {target_mean:.2f} months (observed mean)")
    print(f"Reasonable range: [{target_mean - 2*target_std:.2f}, {target_mean + 2*target_std:.2f}]")
    
    # Clinical knowledge adjustment
    print(f"\nClinical context for {target_col}:")
    if 'PFS' in target_col:
        print("- PFS (Progression-Free Survival) typically ranges 2-20 months in oncology")
        print("- Most common range: 4-12 months")
        print("- Exceptional cases: up to 24+ months")
    
    # Suggested prior
    suggested_mu = target_mean
    suggested_sigma = target_std  # Conservative: allows ±2 std deviation coverage
    
    print(f"\nSUGGESTED INTERCEPT PRIOR:")
    print(f"pm.Normal('intercept', mu={suggested_mu:.1f}, sigma={suggested_sigma:.1f})")
    print(f"This covers range: [{suggested_mu - 2*suggested_sigma:.1f}, {suggested_mu + 2*suggested_sigma:.1f}] with 95% probability")
    
    return suggested_mu, suggested_sigma

# Analyze your data
intercept_mu, intercept_sigma = analyze_target_and_set_priors(bayesian_data, target_col)

## Train Bayesian model with prior

In [ ]:
def build_bayesian_control_model_informed(data, features, target_col, intervention_outcome_col, 
                                        standardize=True, use_informed_priors=True):
    """
    Build Bayesian model with mathematically informed priors and optional standardization
    """
    
    print(f"Building model with standardization={standardize}, informed_priors={use_informed_priors}")
    
    # Prepare data
    model_data = data.copy()
    feature_transforms = {}
    
    if standardize:
        print("Standardizing features...")
        for feature in features:
            if feature in data.columns:
                mean_val = data[feature].mean()
                std_val = data[feature].std()
                model_data[feature] = (data[feature] - mean_val) / std_val
                feature_transforms[feature] = {'mean': mean_val, 'std': std_val}
    
    # Calculate informed priors
    y_mean = data[target_col].mean()
    y_std = data[target_col].std()
    
    if use_informed_priors:
        # Informed priors based on analysis
        if standardize:
            intercept_mu = y_mean
            intercept_sigma = y_std
            coeff_sigma = 0.5 * y_std  # Conservative effect size
        else:
            intercept_mu = y_mean
            intercept_sigma = 2 * y_std
            coeff_sigma = 1.0  # Will be feature-specific in practice
        
        intervention_mu = 0.5
        intervention_sigma = 0.3
    else:
        # Your original priors
        intercept_mu = 6.0
        intercept_sigma = 3.0
        coeff_sigma = 1.0
        intervention_mu = 0.3
        intervention_sigma = 0.5
    
    print(f"Using priors:")
    print(f"  Intercept: Normal({intercept_mu:.1f}, {intercept_sigma:.1f})")
    print(f"  Coefficients: Normal(0, {coeff_sigma:.1f})")
    print(f"  Intervention: Normal({intervention_mu:.1f}, {intervention_sigma:.1f})")
    
    with pm.Model() as model:
        # Intercept
        intercept = pm.Normal('intercept', mu=intercept_mu, sigma=intercept_sigma)
        
        # Feature coefficients
        beta_features = {}
        for feature in features:
            if feature in model_data.columns:
                beta_features[feature] = pm.Normal(f'beta_{feature}', mu=0, sigma=coeff_sigma)
        
        # Intervention coefficient
        beta_intervention = pm.Normal('beta_intervention', mu=intervention_mu, sigma=intervention_sigma)
        
        # Noise
        sigma = pm.HalfNormal('sigma', sigma=y_std)  # Informed noise prior
        
        # Linear combination
        mu = intercept + beta_intervention * model_data[intervention_outcome_col]
        for feature in features:
            if feature in model_data.columns:
                mu += beta_features[feature] * model_data[feature]
        
        # Likelihood
        y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=model_data[target_col])
        
        # Sample
        trace = pm.sample(
            draws=1500, tune=750, target_accept=0.9, 
            random_seed=42, return_inferencedata=True, progressbar=True
        )
        
        # Model assessment
        trace = pm.compute_log_likelihood(trace)
        waic_data = az.waic(trace)
        loo_data = az.loo(trace)
        post_waic, post_loo = waic_data.elpd_waic, loo_data.elpd_loo
        
        print(f"\nModel Assessment:")
        print(f"WAIC: {waic_data.elpd_waic:.2f} ± {waic_data.se:.2f}")
        print(f"LOO: {loo_data.elpd_loo:.2f} ± {loo_data.se:.2f}")
    
    return model, trace, feature_transforms

# Compare models with different prior approaches
print("Building model with standardization and informed priors...")
model_informed, trace_informed, transforms = build_bayesian_control_model_informed(
    bayesian_data, bayesian_features, target_col, intervention_outcome_col,
    standardize=True, use_informed_priors=True
)

## After taking into account priors WAICC and LOO have improved slighlty

```
Prev WAIC -39.64 new -41.71, impact + 5.2%
Prev LOO -40.30 new -43.44, impact + 7.8%
```

# Plot the Posterior Distribution

In [ ]:
ppc = pm.sample_posterior_predictive(trace_informed, model=model_informed)
#ppc = pm.sample_posterior_predictive(bayesian_trace, model=bayesian_model)

In [ ]:
ppc

In [ ]:
y_pred_samples = ppc.posterior_predictive['y_obs'].values.flatten()
y_pred_mean = y_pred_samples.mean()

In [ ]:
plt.hist(y_pred_samples, bins=50, alpha=0.7, density=True, label="Posterior")

plt.axvline(
    y_pred_mean,
    color="blue",
    linestyle="--",
    linewidth=2,
    label=f"ATE bayesian LR: {y_pred_mean:.2f}",
)
plt.title("Posterior - Treatment Coefficient")
plt.xlabel("TE")
plt.ylabel("Density")
plt.legend()

# Inference on different population

In [ ]:
def predict_bayesian(trace, new_data, n_samples=1000):
    """
    For a given new_data dict (with keys: intervention_outcome and bayesian_features),
    sample from the posterior to simulate a predictive distribution.
    """
    posterior = trace.posterior
    n_chains = posterior.dims['chain']
    n_draws = posterior.dims['draw']
    predictions = []
    
    for i in range(n_samples):
        chain_idx = np.random.randint(0, n_chains)
        draw_idx = np.random.randint(0, n_draws)
        # Base linear predictor: intercept + beta_intervention * intervention_outcome
        intercept_val = posterior['intercept'][chain_idx, draw_idx].values
        beta_int_val = posterior['beta_intervention'][chain_idx, draw_idx].values
        pred = intercept_val + beta_int_val * new_data[intervention_outcome_col]
        # Add contribution from each feature
        for f in bayesian_features:
            beta_f = posterior[f'beta_{f}'][chain_idx, draw_idx].values
            pred += beta_f * new_data[f]
        # Add noise
        sigma_val = posterior['sigma'][chain_idx, draw_idx].values
        pred_sample = np.random.normal(pred, sigma_val)
        predictions.append(pred_sample)
    return np.array(predictions)

In [ ]:
bayesian_features

In [ ]:
%autoreload 2
from utilities.bnn_utils import predict_bayesian_with_transforms

# Overall population

In [ ]:
# Define the columns we need to build a new data point:
cols = [intervention_outcome_col] + bayesian_features

# Overall new data point: take the mean (across the control arm data) for each column
overall_new_data = bayesian_data[cols].mean().to_dict()
overall_new_data

#  Posterior of Subpopulation (Age)

In [ ]:
# Split into young and old subpopulations based on the 'age_median' value
median_age = bayesian_data['age_median'].median()
young_new_data = bayesian_data[bayesian_data['age_median'] < median_age][cols].mean().to_dict()
old_new_data   = bayesian_data[bayesian_data['age_median'] >= median_age][cols].mean().to_dict()


In [ ]:
young_new_data

# Option 1: Use original trace (no standardization)

In [ ]:
# Calculate the predictive distributions using the posterior trace (bayesian_trace_v2)
overall_predictions = predict_bayesian(bayesian_trace, overall_new_data, n_samples=1000)
young_predictions   = predict_bayesian(bayesian_trace, young_new_data, n_samples=1000)
old_predictions     = predict_bayesian(bayesian_trace, old_new_data, n_samples=1000)


In [ ]:
# Display summary statistics
print("Overall Predictions:")
print(f"Mean: {np.mean(overall_predictions):.2f}, Std: {np.std(overall_predictions):.2f}")
print(f"95% Credible Interval: [{np.percentile(overall_predictions, 2.5):.2f}, {np.percentile(overall_predictions, 97.5):.2f}]")

print("\nYoung Subpopulation Predictions:")
print(f"Mean: {np.mean(young_predictions):.2f}, Std: {np.std(young_predictions):.2f}")
print(f"95% Credible Interval: [{np.percentile(young_predictions, 2.5):.2f}, {np.percentile(young_predictions, 97.5):.2f}]")

print("\nOld Subpopulation Predictions:")
print(f"Mean: {np.mean(old_predictions):.2f}, Std: {np.std(old_predictions):.2f}")
print(f"95% Credible Interval: [{np.percentile(old_predictions, 2.5):.2f}, {np.percentile(old_predictions, 97.5):.2f}]")


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.hist(young_predictions, bins=50, alpha=0.6, label='Young Subpopulation', density=True, color='blue')
ax.hist(old_predictions, bins=50, alpha=0.6, label='Old Subpopulation', density=True, color='red')

ax.set_xlabel('Predicted Outcome')
ax.set_ylabel('Density')
ax.set_title('Posterior Predictive Distributions by Age Group')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate means for each distribution
overall_mean = np.mean(overall_predictions)
young_mean = np.mean(young_predictions)
old_mean = np.mean(old_predictions)

# Visualization of posterior distributions
fig, ax = plt.subplots(1, 3, figsize=(24, 8))

ax[0].hist(overall_predictions, bins=50, density=True, alpha=0.7, color='skyblue')
ax[0].axvline(overall_mean, color='darkblue', linestyle='--', linewidth=2, label=f'Mean: {overall_mean:.2f}')
ax[0].set_title(f"Overall Posterior Distribution (μ = {overall_mean:.2f})")
ax[0].set_xlabel("Predicted Outcome")
ax[0].set_ylabel("Density")
ax[0].legend()

ax[1].hist(young_predictions, bins=50, density=True, alpha=0.7, color='green')
ax[1].axvline(young_mean, color='darkgreen', linestyle='--', linewidth=2, label=f'Mean: {young_mean:.2f}')
ax[1].set_title(f"Young Subpopulation Posterior (μ = {young_mean:.2f})")
ax[1].set_xlabel("Predicted Outcome")
ax[1].legend()

ax[2].hist(old_predictions, bins=50, density=True, alpha=0.7, color='red')
ax[2].axvline(old_mean, color='darkred', linestyle='--', linewidth=2, label=f'Mean: {old_mean:.2f}')
ax[2].set_title(f"Old Subpopulation Posterior (μ = {old_mean:.2f})")
ax[2].set_xlabel("Predicted Outcome")
ax[2].legend()

plt.tight_layout()
plt.show()

# Option 2: Use informed trace (with standardization)


In [ ]:
print("Using informed trace (with standardization):")
overall_predictions_informed = predict_bayesian_with_transforms(trace_informed, overall_new_data, transforms, n_samples=1000, intervention_outcome_col=intervention_outcome_col, bayesian_features=bayesian_features)
young_predictions_informed = predict_bayesian_with_transforms(trace_informed, young_new_data, transforms, n_samples=1000, intervention_outcome_col=intervention_outcome_col, bayesian_features=bayesian_features)
old_predictions_informed = predict_bayesian_with_transforms(trace_informed, old_new_data, transforms, n_samples=1000, intervention_outcome_col=intervention_outcome_col, bayesian_features=bayesian_features)


In [ ]:
print(f"\nInformed model predictions:")
print(f"Overall mean: {np.mean(overall_predictions_informed):.2f}")
print(f"Young mean: {np.mean(young_predictions_informed):.2f}")
print(f"Old mean: {np.mean(old_predictions_informed):.2f}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.hist(young_predictions_informed, bins=50, alpha=0.6, label='Young Subpopulation', density=True, color='blue')
ax.hist(old_predictions_informed, bins=50, alpha=0.6, label='Old Subpopulation', density=True, color='red')

ax.set_xlabel('Predicted Outcome')
ax.set_ylabel('Density')
ax.set_title('Posterior Predictive Distributions by Age Group')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#  Posterior of Subpopulation (Disease Stage four)

In [ ]:
# Create new data for disease_stage_IV subpopulations: one for values < 90 and another for values ≥ 90
low_dsiv_new_data = bayesian_data[bayesian_data['disease_stage_IV'] < 90][cols].mean().to_dict()
high_dsiv_new_data = bayesian_data[bayesian_data['disease_stage_IV'] >= 90][cols].mean().to_dict()

# Calculate the predictive distributions using the posterior trace (bayesian_trace_v2)
low_dsiv_predictions = predict_bayesian(bayesian_trace, low_dsiv_new_data, n_samples=1000)
high_dsiv_predictions = predict_bayesian(bayesian_trace, high_dsiv_new_data, n_samples=1000)

# Plot the predictive distributions for the disease_stage_IV subpopulations
fig, ax = plt.subplots(figsize=(8, 6))

ax.hist(low_dsiv_predictions, bins=50, alpha=0.6, label='Disease Stage IV < 90', density=True, color='blue')
ax.hist(high_dsiv_predictions, bins=50, alpha=0.6, label='Disease Stage IV ≥ 90', density=True, color='red')

ax.set_xlabel('Predicted Outcome')
ax.set_ylabel('Density')
ax.set_title('Posterior Predictive Distributions by Disease Stage IV')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Print summary statistics for disease_stage_IV subpopulation predictions
print("\nDisease Stage IV < 90 Predictions:")
print(f"Mean: {np.mean(low_dsiv_predictions):.2f}, Std: {np.std(low_dsiv_predictions):.2f}")
print(f"95% Credible Interval: [{np.percentile(low_dsiv_predictions, 2.5):.2f}, {np.percentile(low_dsiv_predictions, 97.5):.2f}]")

print("\nDisease Stage IV ≥ 90 Predictions:")
print(f"Mean: {np.mean(high_dsiv_predictions):.2f}, Std: {np.std(high_dsiv_predictions):.2f}")
print(f"95% Credible Interval: [{np.percentile(high_dsiv_predictions, 2.5):.2f}, {np.percentile(high_dsiv_predictions, 97.5):.2f}]")

# Target ATE

In [ ]:
def calculate_ate_probability(trace, subpop_data, intervention_outcome, target_ate, n_samples=1000):
    """
    Calculate probability of achieving target ATE for a subpopulation
    
    Parameters:
    - trace: Bayesian model trace
    - subpop_data: Dictionary with subpopulation characteristics
    - intervention_outcome: Expected intervention arm outcome
    - target_ate: Target ATE value we want to achieve
    - n_samples: Number of posterior samples
    
    Returns:
    - prob_achieve: Probability of achieving target ATE
    - ate_samples: Array of ATE samples for further analysis
    """
    
    # Get control arm predictions for subpopulation
    control_predictions = predict_bayesian(trace, subpop_data, n_samples=n_samples)
    
    # Calculate ATE samples: intervention - control
    ate_samples = intervention_outcome - control_predictions
    
    # Calculate probability of achieving target ATE
    prob_achieve = np.mean(ate_samples >= target_ate)
    
    return prob_achieve, ate_samples

# Example usage with your existing data
target_ate = 0.5

# Calculate for young subpopulation
prob_young, ate_young = calculate_ate_probability(
    bayesian_trace, young_new_data, 
    young_new_data[intervention_outcome_col], target_ate
)

# Calculate for old subpopulation  
prob_old, ate_old = calculate_ate_probability(
    bayesian_trace, old_new_data,
    old_new_data[intervention_outcome_col], target_ate
)

print(f"Probability of ATE >= {target_ate}:")
print(f"Young subpopulation: {prob_young:.3f}")
print(f"Old subpopulation: {prob_old:.3f}")

# Bayesian Neural Network

In [ ]:
%autoreload 2

In [ ]:
from utilities.bnn_utils import build_bnn_with_log_target, predict_log_transform_bnn, build_bayesian_neural_network, predict_bnn_with_transforms, compare_linear_vs_bnn_predictions

## Training on log of outcomes
With one hidden layer and 25 no. of hidden neurons

In [ ]:
# 1. Log-transform approach
print("\n1. LOG-TRANSFORM APPROACH")
print("-" * 40)
model_log, trace_log, transforms_log = build_bnn_with_log_target(
    bayesian_data, bayesian_features, target_col, intervention_outcome_col, hidden_units=[25]
)
preds_log = predict_log_transform_bnn(
    trace_log, overall_new_data, transforms_log, bayesian_features, 
    intervention_outcome_col, [25], n_samples=1000
)


print(f"Mean: {np.mean(preds_log):.2f}, Std: {np.std(preds_log):.2f}")
print(f"95% Credible Interval: [{np.percentile(preds_log, 2.5):.2f}, {np.percentile(preds_log, 97.5):.2f}]")

## Building a smaller netowrk
Since amount of data is small, we reduced the network size and no. of hidden units

In [ ]:
bnn_model, bnn_trace, bnn_transforms = build_bayesian_neural_network(
        bayesian_data, bayesian_features, target_col, intervention_outcome_col,
        hidden_units=[8],  # Smaller network for limited data
        standardize=True, 
        use_informed_priors=True
)

In [ ]:
%autoreload 2

In [ ]:
# New data
overall_comparison = compare_linear_vs_bnn_predictions(
        trace_informed, bnn_trace, bnn_transforms, overall_new_data,
        bayesian_features, intervention_outcome_col, transforms, hidden_units=[8]
)

print("="*60)
print("BAYESIAN MODEL COMPARISON - OVERALL POPULATION")
print("="*60)

# Bayesian Linear Regression Results
print("\n📊 BAYESIAN LINEAR REGRESSION:")
print("-" * 40)
print(f"Mean: {np.mean(overall_comparison['linear']):.2f} months")
print(f"Std:  {np.std(overall_comparison['linear']):.2f} months")
print(f"95% Credible Interval: [{np.percentile(overall_comparison['linear'], 2.5):.2f}, {np.percentile(overall_comparison['linear'], 97.5):.2f}] months")
print(f"90% Credible Interval: [{np.percentile(overall_comparison['linear'], 5):.2f}, {np.percentile(overall_comparison['linear'], 95):.2f}] months")

# Bayesian Neural Network Results  
print("\n🧠 BAYESIAN NEURAL NETWORK:")
print("-" * 40)
print(f"Mean: {np.mean(overall_comparison['bnn']):.2f} months")
print(f"Std:  {np.std(overall_comparison['bnn']):.2f} months")
print(f"95% Credible Interval: [{np.percentile(overall_comparison['bnn'], 2.5):.2f}, {np.percentile(overall_comparison['bnn'], 97.5):.2f}] months")
print(f"90% Credible Interval: [{np.percentile(overall_comparison['bnn'], 5):.2f}, {np.percentile(overall_comparison['bnn'], 95):.2f}] months")

# Model Comparison
print("\n🔍 MODEL COMPARISON:")
print("-" * 40)
mean_diff = np.mean(overall_comparison['bnn']) - np.mean(overall_comparison['linear'])
std_diff = np.std(overall_comparison['bnn']) - np.std(overall_comparison['linear'])
print(f"Mean difference (BNN - LR): {mean_diff:+.2f} months")
print(f"Std difference (BNN - LR):  {std_diff:+.2f} months")



## Compare  distributions of Bayesian NN vs Bayesian LR

### Young Subpopulation

In [ ]:
# Young population comparison
young_comparison = compare_linear_vs_bnn_predictions(
    trace_informed, bnn_trace, bnn_transforms, young_new_data,
    bayesian_features, intervention_outcome_col, transforms, hidden_units=[8]
)

print("="*60)
print("BAYESIAN MODEL COMPARISON - YOUNG SUBPOPULATION")
print("="*60)

# Bayesian Linear Regression Results
print("\n📊 BAYESIAN LINEAR REGRESSION (Young):")
print("-" * 40)
print(f"Mean: {np.mean(young_comparison['linear']):.2f} months")
print(f"Std:  {np.std(young_comparison['linear']):.2f} months")
print(f"95% Credible Interval: [{np.percentile(young_comparison['linear'], 2.5):.2f}, {np.percentile(young_comparison['linear'], 97.5):.2f}] months")
print(f"90% Credible Interval: [{np.percentile(young_comparison['linear'], 5):.2f}, {np.percentile(young_comparison['linear'], 95):.2f}] months")

# Bayesian Neural Network Results  
print("\n🧠 BAYESIAN NEURAL NETWORK (Young):")
print("-" * 40)
print(f"Mean: {np.mean(young_comparison['bnn']):.2f} months")
print(f"Std:  {np.std(young_comparison['bnn']):.2f} months")
print(f"95% Credible Interval: [{np.percentile(young_comparison['bnn'], 2.5):.2f}, {np.percentile(young_comparison['bnn'], 97.5):.2f}] months")
print(f"90% Credible Interval: [{np.percentile(young_comparison['bnn'], 5):.2f}, {np.percentile(young_comparison['bnn'], 95):.2f}] months")

# Model Comparison
print("\n🔍 MODEL COMPARISON (Young):")
print("-" * 40)
mean_diff_young = np.mean(young_comparison['bnn']) - np.mean(young_comparison['linear'])
std_diff_young = np.std(young_comparison['bnn']) - np.std(young_comparison['linear'])
print(f"Mean difference (BNN - LR): {mean_diff_young:+.2f} months")
print(f"Std difference (BNN - LR):  {std_diff_young:+.2f} months")

# Additional diagnostics
print(f"\nNegative predictions (Young):")
print(f"  Linear: {(young_comparison['linear'] < 0).mean():.1%}")
print(f"  BNN:    {(young_comparison['bnn'] < 0).mean():.1%}")

print(f"\nPredictions > 12 months (Young):")
print(f"  Linear: {(young_comparison['linear'] > 12).mean():.1%}")
print(f"  BNN:    {(young_comparison['bnn'] > 12).mean():.1%}")

# Create separate plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Overlapping distributions
axes[0, 0].hist(young_comparison['linear'], bins=50, alpha=0.6, label='Bayesian Linear Regression', 
                density=True, color='blue')
axes[0, 0].hist(young_comparison['bnn'], bins=50, alpha=0.6, label='Bayesian Neural Network', 
                density=True, color='red')
axes[0, 0].axvline(np.mean(young_comparison['linear']), color='blue', linestyle='--', 
                   label=f'Linear Mean: {np.mean(young_comparison["linear"]):.2f}')
axes[0, 0].axvline(np.mean(young_comparison['bnn']), color='red', linestyle='--', 
                   label=f'BNN Mean: {np.mean(young_comparison["bnn"]):.2f}')
axes[0, 0].set_xlabel('Predicted PFS (months)')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Young Subpopulation: Model Comparison')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Linear model only
axes[0, 1].hist(young_comparison['linear'], bins=50, alpha=0.7, density=True, color='blue')
axes[0, 1].axvline(np.mean(young_comparison['linear']), color='darkblue', linestyle='--', linewidth=2, 
                   label=f'Mean: {np.mean(young_comparison["linear"]):.2f}')
axes[0, 1].set_xlabel('Predicted PFS (months)')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Young Subpopulation: Bayesian Linear Regression')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: BNN model only
axes[1, 0].hist(young_comparison['bnn'], bins=50, alpha=0.7, density=True, color='red')
axes[1, 0].axvline(np.mean(young_comparison['bnn']), color='darkred', linestyle='--', linewidth=2, 
                   label=f'Mean: {np.mean(young_comparison["bnn"]):.2f}')
axes[1, 0].set_xlabel('Predicted PFS (months)')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Young Subpopulation: Bayesian Neural Network')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Box plot comparison
box_data = [young_comparison['linear'], young_comparison['bnn']]
box_labels = ['Linear Regression', 'Neural Network']
box_plot = axes[1, 1].boxplot(box_data, labels=box_labels, patch_artist=True)

# Color the boxes
colors = ['lightblue', 'lightcoral']
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)

axes[1, 1].set_ylabel('Predicted PFS (months)')
axes[1, 1].set_title('Young Subpopulation: Box Plot Comparison')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*60)
print("YOUNG SUBPOPULATION SUMMARY TABLE")
print("="*60)
print(f"{'Model':<20} {'Mean':<8} {'Std':<8} {'95% CI':<20} {'Neg %':<8} {'>12mo %':<8}")
print("-" * 70)
print(f"{'Linear Regression':<20} {np.mean(young_comparison['linear']):<8.2f} {np.std(young_comparison['linear']):<8.2f} [{np.percentile(young_comparison['linear'], 2.5):.2f}, {np.percentile(young_comparison['linear'], 97.5):.2f}]     {(young_comparison['linear'] < 0).mean()*100:<8.1f} {(young_comparison['linear'] > 12).mean()*100:<8.1f}")
print(f"{'Neural Network':<20} {np.mean(young_comparison['bnn']):<8.2f} {np.std(young_comparison['bnn']):<8.2f} [{np.percentile(young_comparison['bnn'], 2.5):.2f}, {np.percentile(young_comparison['bnn'], 97.5):.2f}]     {(young_comparison['bnn'] < 0).mean()*100:<8.1f} {(young_comparison['bnn'] > 12).mean()*100:<8.1f}")

#### Old population - Bayesian LR vs Bayesian NN

In [ ]:
# Old population comparison
old_comparison = compare_linear_vs_bnn_predictions(
    trace_informed, bnn_trace, bnn_transforms, old_new_data,
    bayesian_features, intervention_outcome_col, transforms, hidden_units=[8]
)

print("\n" + "="*60)
print("BAYESIAN MODEL COMPARISON - OLD SUBPOPULATION")
print("="*60)

# Print similar statistics for old population
print("\n📊 BAYESIAN LINEAR REGRESSION (Old):")
print("-" * 40)
print(f"Mean: {np.mean(old_comparison['linear']):.2f} months")
print(f"Std:  {np.std(old_comparison['linear']):.2f} months")
print(f"95% Credible Interval: [{np.percentile(old_comparison['linear'], 2.5):.2f}, {np.percentile(old_comparison['linear'], 97.5):.2f}] months")

print("\n🧠 BAYESIAN NEURAL NETWORK (Old):")
print("-" * 40)
print(f"Mean: {np.mean(old_comparison['bnn']):.2f} months")
print(f"Std:  {np.std(old_comparison['bnn']):.2f} months")
print(f"95% Credible Interval: [{np.percentile(old_comparison['bnn'], 2.5):.2f}, {np.percentile(old_comparison['bnn'], 97.5):.2f}] months")


### Bayesian NN - Young vs Old Populations

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Young vs Old - Linear Regression
axes[0, 0].hist(young_comparison['linear'], bins=40, alpha=0.6, label='Young Subpopulation', 
                density=True, color='blue')
axes[0, 0].hist(old_comparison['linear'], bins=40, alpha=0.6, label='Old Subpopulation', 
                density=True, color='orange')
axes[0, 0].axvline(np.mean(young_comparison['linear']), color='blue', linestyle='--', linewidth=2,
                   label=f'Young Mean: {np.mean(young_comparison["linear"]):.2f}')
axes[0, 0].axvline(np.mean(old_comparison['linear']), color='orange', linestyle='--', linewidth=2,
                   label=f'Old Mean: {np.mean(old_comparison["linear"]):.2f}')
axes[0, 0].set_xlabel('Predicted PFS (months)')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('📊 Bayesian Linear Regression: Young vs Old')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Young vs Old - Bayesian Neural Network
axes[0, 1].hist(young_comparison['bnn'], bins=40, alpha=0.6, label='Young Subpopulation', 
                density=True, color='green')
axes[0, 1].hist(old_comparison['bnn'], bins=40, alpha=0.6, label='Old Subpopulation', 
                density=True, color='red')
axes[0, 1].axvline(np.mean(young_comparison['bnn']), color='green', linestyle='--', linewidth=2,
                   label=f'Young Mean: {np.mean(young_comparison["bnn"]):.2f}')
axes[0, 1].axvline(np.mean(old_comparison['bnn']), color='red', linestyle='--', linewidth=2,
                   label=f'Old Mean: {np.mean(old_comparison["bnn"]):.2f}')
axes[0, 1].set_xlabel('Predicted PFS (months)')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('🧠 Bayesian Neural Network: Young vs Old')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Young Population - Model Comparison
axes[1, 0].hist(young_comparison['linear'], bins=35, alpha=0.6, label='Linear Regression', 
                density=True, color='blue')
axes[1, 0].hist(young_comparison['bnn'], bins=35, alpha=0.6, label='Neural Network', 
                density=True, color='green')
axes[1, 0].axvline(np.mean(young_comparison['linear']), color='blue', linestyle='--', linewidth=2,
                   label=f'LR Mean: {np.mean(young_comparison["linear"]):.2f}')
axes[1, 0].axvline(np.mean(young_comparison['bnn']), color='green', linestyle='--', linewidth=2,
                   label=f'BNN Mean: {np.mean(young_comparison["bnn"]):.2f}')
axes[1, 0].set_xlabel('Predicted PFS (months)')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Young Subpopulation: LR vs BNN')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Old Population - Model Comparison
axes[1, 1].hist(old_comparison['linear'], bins=35, alpha=0.6, label='Linear Regression', 
                density=True, color='orange')
axes[1, 1].hist(old_comparison['bnn'], bins=35, alpha=0.6, label='Neural Network', 
                density=True, color='red')
axes[1, 1].axvline(np.mean(old_comparison['linear']), color='orange', linestyle='--', linewidth=2,
                   label=f'LR Mean: {np.mean(old_comparison["linear"]):.2f}')
axes[1, 1].axvline(np.mean(old_comparison['bnn']), color='red', linestyle='--', linewidth=2,
                   label=f'BNN Mean: {np.mean(old_comparison["bnn"]):.2f}')
axes[1, 1].set_xlabel('Predicted PFS (months)')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Old Subpopulation: LR vs BNN')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot Posterior Distribution - BNN

In [ ]:
def plot_bnn_posterior_predictive():
    """
    Plot posterior predictive distribution for BNN model
    """
    
    ppc_bnn = pm.sample_posterior_predictive(bnn_trace, model=bnn_model)
    y_pred_samples = ppc_bnn.posterior_predictive['y_obs'].values.flatten()
    y_pred_mean = y_pred_samples.mean()
    plt.hist(y_pred_samples, bins=50, alpha=0.7, density=True, label="Posterior")

    plt.axvline(
        y_pred_mean,
        color="blue",
        linestyle="--",
        linewidth=2,
        label=f"ATE bayesian BNN: {y_pred_mean:.2f}",
    )
    plt.title("Posterior - Treatment Coefficient")
    plt.xlabel("TE")
    plt.ylabel("Density")
    plt.legend()

plot_bnn_posterior_predictive()



# Gaussian Process

In [ ]:
%autoreload 2

In [ ]:
from utilities.gp_utils import build_gaussian_process, predict_gp, plot_gp_predictions, compute_kernel_matrix

In [ ]:
gp_model, gp_trace, gp_params = build_gaussian_process(
    bayesian_data,
    bayesian_features,
    target_col,
    intervention_outcome_col,
    kernel='matern52',
    standardize=True,
    use_log_target=True  # Set to True to ensure positive predictions
)

# Make predictions
young_preds = predict_gp(
    gp_trace, gp_params, young_new_data,
    bayesian_features, intervention_outcome_col,
    n_samples=2000, return_std=False
)

print(f"Young Population GP Predictions:")
print(f"Mean: {np.mean(young_preds):.2f}, Std: {np.std(young_preds):.2f}")
print(f"95% CI: [{np.percentile(young_preds, 2.5):.2f}, {np.percentile(young_preds, 97.5):.2f}]")

In [ ]:
plot_gp_predictions(young_predictions, title="Young Population - GP Predictions")

# Point Estimatation
Leave-one out protocol to get R2, RMSE values and compare with other tree based models




In [ ]:
bayesian_features

In [ ]:
list_ftr_selected = bayesian_features.copy()

### Inference  using vectorized operations

We could use ```predict_bayesian``` function as an alternative if dataset is small. But for the sake of demonstration of approach we use ```predict_vectorized```


In [ ]:
def predict_vectorized(trace, new_data_scaled, bayesian_features, n_samples=1000):
    """
    Vectorized prediction - returns 1D array using dynamic Bayesian features.
    
    For each feature in bayesian_features, the corresponding parameter in the posterior
    is assumed to be beta_{feature}.
    """
    posterior_samples = trace.posterior

    # Randomly sample indices from the posterior
    n_chains, n_draws = posterior_samples.dims['chain'], posterior_samples.dims['draw']
    chain_indices = np.random.randint(0, n_chains, n_samples)
    draw_indices = np.random.randint(0, n_draws, n_samples)

    # Extract the intercept and sigma samples
    intercept_samples = posterior_samples['intercept'][chain_indices, draw_indices].values.flatten()
    sigma_samples = posterior_samples['sigma'][chain_indices, draw_indices].values.flatten()

    # Initialize the linear predictor with the intercept
    mu_pred = intercept_samples.copy()
    
    # Add contribution from each feature dynamically
    for feature in bayesian_features:
        param_name = f"beta_{feature}"
        beta_samples = posterior_samples[param_name][chain_indices, draw_indices].values.flatten()
        mu_pred += beta_samples * new_data_scaled[feature]
    
    # Add noise for the final prediction
    predictions = np.random.normal(mu_pred, sigma_samples)
    
    return predictions  # Returns a 1D numpy array of shape (n_samples,)

# Run leave-one-out Bayesian inference for control arm prediction


In [ ]:
def build_bayesian_model_fast(data, features, target_col, intervention_outcome_col, 
                             draws=500, tune=250):
    """
    Build and sample Bayesian model with custom sampling parameters for leave-one-out inference.
    Returns only the trace for prediction.
    """
    
    with pm.Model() as model:
        # Same priors as original build_bayesian_control_model
        intercept = pm.Normal('intercept', mu=0, sigma=10.0)
        
        # Feature coefficients
        beta_features = {}
        for feature in features:
            if feature in data.columns:
                beta_features[feature] = pm.Normal(f'beta_{feature}', mu=0, sigma=1.0)
        
        # Intervention coefficient
        beta_intervention = pm.Normal('beta_intervention', mu=0.3, sigma=0.5)
        
        # Noise parameter
        sigma = pm.HalfNormal('sigma', sigma=2.0)
        
        # Linear combination
        mu = intercept + beta_intervention * data[intervention_outcome_col]
        for feature in features:
            if feature in data.columns:
                mu += beta_features[feature] * data[feature]
        
        # Likelihood
        y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=data[target_col])
        
        # Sample with custom parameters
        trace = pm.sample(
            draws=draws, tune=tune, target_accept=0.9, 
            random_seed=42, return_inferencedata=True, progressbar=False, verbose=False, 
        )
    
    return trace

In [ ]:
# Hide all pymc related logs

import logging
import warnings

logger = logging.getLogger("pymc")
logger.setLevel(logging.ERROR)
# suppress all FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning)


## Bayesian LR

In [ ]:
#with pred CI and actual CI for OUTCOME and ATE

results_bayesian = []
bayesian_draws = 500
bayesian_tune = 250

for index, row in training_df.iterrows():
    rct_name = row[trial_id_col]
    is_arm_control = row[control_arm_col]
    
    if is_arm_control == 1:
        # Ground truth
        outcome_control = round(row[target_col], 2)
        
        # ==========================================
        # COLLECT ACTUAL CI DATA FOR IoU CALCULATION
        # ==========================================
        
        # Get the actual CI for the current control arm (for IoU calculation)
        actual_outcome_ci = None
        if "PFS_median_CI" in training_df.columns:
            actual_ci_value = row["PFS_median_CI"]
            if pd.notna(actual_ci_value) and actual_ci_value is not None:
                # Handle different formats of CI storage
                if isinstance(actual_ci_value, (list, tuple, np.ndarray)):
                    actual_outcome_ci = actual_ci_value
                elif isinstance(actual_ci_value, str):
                    try:
                        # Try to parse string representation like "[x, y]"
                        actual_outcome_ci = eval(actual_ci_value)
                    except:
                        print(f"Warning: Could not parse CI string for {rct_name}: {actual_ci_value}")
                        actual_outcome_ci = None
                else:
                    actual_outcome_ci = None
        
        # Get the trt outcome of the RCT targeted
        trt_arm = training_df.loc[training_df[trial_id_col] == rct_name, :]
        trt_outcome = trt_arm.loc[trt_arm[control_arm_col] != 1, target_col]
        if trt_outcome.empty or pd.isna(trt_outcome.mean()):
            continue
        trt_outcome = round(trt_outcome.mean(), 2)
        
        # Prepare training data
        training_j = training_df.copy()
        # Flag inference row
        training_j["is_to_predict"] = np.where(training_j.index == index, 1, 0)
        # Flag RCT row
        training_j["is_targeted_rct"] = np.where(training_j[trial_id_col] == rct_name, 1, 0)
        
        # Add intervention outcome as a feature for all control arms
        for idx, control_row in training_j[training_j[control_arm_col] == 1].iterrows():
            ctrl_rct = control_row[trial_id_col]
            # Get intervention outcome for this control's trial
            ctrl_trt_outcome = training_j.loc[(training_j[trial_id_col] == ctrl_rct) & 
                                             (training_j[control_arm_col] != 1), target_col]
            
            if not ctrl_trt_outcome.empty and not pd.isna(ctrl_trt_outcome.mean()):
                training_j.loc[idx, intervention_outcome_col] = round(ctrl_trt_outcome.mean(), 2)
            else:
                training_j.loc[idx, intervention_outcome_col] = np.nan
        
        training_j = training_j.dropna(subset=[intervention_outcome_col])
        
        # Only consider control arms for training
        training_j = training_j.loc[training_j[control_arm_col] == 1, :]
        training_j = prepare_data(training_j, list_ftr_selected + ["is_to_predict", "is_targeted_rct", intervention_outcome_col, target_col])
        training_j = run_fillna(training_j)
        
        # Make sure the target row has the intervention outcome
        training_j.loc[training_j["is_to_predict"] == 1, intervention_outcome_col] = trt_outcome
        
        # Extract inference data
        inference_df = training_j.loc[training_j["is_to_predict"] == 1, :]
        if inference_df.empty:
            print(f"Skipping {rct_name} as no inference data available.")
            continue
            
        # Remove inference row and targeted RCT from training
        training_subset = training_j.loc[training_j["is_to_predict"] != 1, :]
        training_subset = training_subset.loc[training_subset["is_targeted_rct"] != 1, :]
        
        # Check if we have enough training data
        if training_subset.shape[0] < 10:
            print(f"Skipping {rct_name} due to insufficient data for Bayesian inference.")
            continue
        
        try:
            # ==========================================
            # CALCULATE TRAINING ACCURACY METRICS
            # ==========================================
            
            # Build and sample Bayesian model
            bayesian_trace = build_bayesian_model_fast(
                training_subset, list_ftr_selected, target_col, intervention_outcome_col,
                draws=bayesian_draws, tune=bayesian_tune
            )
            
            # Get training predictions for evaluation
            training_preds = []
            for _, train_row in training_subset.iterrows():
                train_data = train_row[list_ftr_selected + [intervention_outcome_col]].to_dict()
                train_pred_samples = predict_bayesian(bayesian_trace, train_data, n_samples=500)
                training_preds.append(np.mean(train_pred_samples))
            
            training_preds = np.array(training_preds)
            training_true = training_subset[target_col].values
            
            # Calculate training metrics
            training_rmse = round(np.sqrt(mean_squared_error(training_true, training_preds)), 3)
            training_r2 = round(r2_score(training_true, training_preds), 3)
            training_mae = round(np.mean(np.abs(training_true - training_preds)), 3)
            
            print(f"  Training metrics - RMSE: {training_rmse}, R²: {training_r2}, MAE: {training_mae}")
            
            # ==========================================
            # GET TEST PREDICTION WITH UNCERTAINTY
            # ==========================================
            
            # Create prediction data
            new_data = inference_df[list_ftr_selected + [intervention_outcome_col]].iloc[0].to_dict()
            
            # Make predictions with full posterior distribution
            pred_samples = predict_bayesian(bayesian_trace, new_data, n_samples=2000)
            
            # Calculate prediction statistics
            predicted_outcome = round(np.nanmean(pred_samples), 2)
            pred_std = round(np.nanstd(pred_samples), 2)
            pred_median = round(np.nanmedian(pred_samples), 2)
            
            # Calculate confidence intervals
            pred_ci_95 = [
                round(np.percentile(pred_samples, 2.5), 2),
                round(np.percentile(pred_samples, 97.5), 2)
            ]
            pred_ci_90 = [
                round(np.percentile(pred_samples, 5.0), 2),
                round(np.percentile(pred_samples, 95.0), 2)
            ]
            pred_ci_80 = [
                round(np.percentile(pred_samples, 10.0), 2),
                round(np.percentile(pred_samples, 90.0), 2)
            ]
            
            # Additional uncertainty measures
            pred_ci_width = round(pred_ci_95[1] - pred_ci_95[0], 3)
            pred_iqr = round(np.percentile(pred_samples, 75) - np.percentile(pred_samples, 25), 3)
            
            # ==========================================
            # CALCULATE ATE DISTRIBUTION
            # ==========================================
            
            # Calculate ATEs
            real_ate = round(trt_outcome - outcome_control, 2)
            pred_ate = round(trt_outcome - predicted_outcome, 2)
            
            # Calculate ATE distribution (treatment outcome - predicted control distribution)
            ate_samples = trt_outcome - pred_samples
            ate_mean = round(np.mean(ate_samples), 2)
            ate_std = round(np.std(ate_samples), 2)
            ate_ci_95 = [
                round(np.percentile(ate_samples, 2.5), 2),
                round(np.percentile(ate_samples, 97.5), 2)
            ]
            ate_ci_90 = [
                round(np.percentile(ate_samples, 5.0), 2),
                round(np.percentile(ate_samples, 95.0), 2)
            ]
            ate_ci_width = round(ate_ci_95[1] - ate_ci_95[0], 3)
            
            # Probability of positive ATE
            prob_positive_ate = round(np.mean(ate_samples > 0), 3)
            
            # Empirical-based ATE metrics (alternative calculation)
            ate_ci_95_empirical = [
                round(trt_outcome - pred_ci_95[1], 2),  # Note: reversed order
                round(trt_outcome - pred_ci_95[0], 2)
            ]
            ate_ci_90_empirical = [
                round(trt_outcome - pred_ci_90[1], 2),
                round(trt_outcome - pred_ci_90[0], 2)
            ]
            
            # Alternative probability calculation using empirical quantiles
            if pred_ci_95[1] < trt_outcome:
                prob_positive_ate_empirical = 1.0
            elif pred_ci_95[0] > trt_outcome:
                prob_positive_ate_empirical = 0.0
            else:
                prob_positive_ate_empirical = prob_positive_ate
            
            # ==========================================
            # STORE ENHANCED RESULTS WITH ACTUAL CI
            # ==========================================
            
            results_bayesian.append({
                # Original results
                "real_ate": real_ate,
                "pred_ate": pred_ate,
                "outcome_control": outcome_control,
                "predicted_outcome": predicted_outcome,
                "rct_name": rct_name,
                "intervention": row["intervention"],
                "Arm": row["Arm"],
                "n_training_samples": len(training_subset),
                
                # ATE distribution metrics (needed for performance evaluation)
                "ate_mean": ate_mean,
                "ate_std": ate_std,
                "ate_ci_95": ate_ci_95,
                "ate_ci_90": ate_ci_90,
                "ate_ci_width": ate_ci_width,
                "ate_samples": ate_samples.tolist(),
                "prob_positive_ate": prob_positive_ate,
                
                # Enhanced prediction uncertainty metrics
                "pred_std": pred_std,
                "pred_median": pred_median,
                "pred_ci_95": pred_ci_95,
                "pred_ci_90": pred_ci_90,
                "pred_ci_80": pred_ci_80,
                "pred_ci_width": pred_ci_width,
                "pred_iqr": pred_iqr,
                "pred_samples": pred_samples.tolist(),
                
                # Empirical-based ATE metrics (more accurate)
                "ate_ci_95_empirical": ate_ci_95_empirical,
                "ate_ci_90_empirical": ate_ci_90_empirical,
                "prob_positive_ate_empirical": prob_positive_ate_empirical,
                
                # Training accuracy metrics
                "training_rmse": training_rmse,
                "training_r2": training_r2,
                "training_mae": training_mae,
                
                # Additional training set statistics
                "training_set_std": round(np.std(training_true), 3),
                "training_pred_std": round(np.std(training_preds), 3),
                
                # NEW: Data for IoU calculation
                "actual_outcome_ci": actual_outcome_ci,  # Actual CI from RCT data
                "original_row_index": index,  # Store original index for potential merging
                
                # Additional uncertainty measures for comprehensive evaluation
                "pred_range_90": round(pred_ci_90[1] - pred_ci_90[0], 3),
                "pred_range_80": round(pred_ci_80[1] - pred_ci_80[0], 3),
            })
            
            print(f"{rct_name} - arm: {row['Arm']} - intervention: {trt_outcome}")
            print(f"  Real outcome: {outcome_control}, Pred: {predicted_outcome} ± {pred_std}")
            print(f"  Pred CI 95%: {pred_ci_95}, width: {pred_ci_width}")
            if actual_outcome_ci:
                print(f"  Actual CI: {actual_outcome_ci}")
            print(f"  Real ATE: {real_ate}, Pred ATE: {pred_ate} (95% CI: {ate_ci_95})")
            print(f"  P(ATE > 0): {prob_positive_ate} (empirical: {prob_positive_ate_empirical})")
            
        except Exception as e:
            print(f"Error with {rct_name}: {str(e)}")
            continue



In [ ]:
results_bayesian_df = pd.DataFrame(results_bayesian)
results_bayesian_df.head(2)

## Bayesian Neural Network

In [ ]:
import pytensor.tensor as pt

def build_bnn_model_fast_loo(data, features, target_col, intervention_outcome_col, 
                            hidden_units=[8], draws=500, tune=250):
    """
    Build and sample BNN model with custom sampling parameters for leave-one-out inference.
    Returns the trace and feature transforms for prediction.
    """
    
    # Standardize features for BNN
    model_data = data.copy()
    feature_transforms = {}
    
    # Standardize input features
    for feature in features:
        if feature in data.columns:
            mean_val = data[feature].mean()
            std_val = data[feature].std()
            if std_val > 0:
                model_data[feature] = (data[feature] - mean_val) / std_val
            else:
                model_data[feature] = data[feature] - mean_val
            feature_transforms[feature] = {
                "mean": mean_val,
                "std": std_val if std_val > 0 else 1.0,
            }
    
    # Standardize intervention outcome
    int_mean = data[intervention_outcome_col].mean()
    int_std = data[intervention_outcome_col].std()
    if int_std > 0:
        model_data[intervention_outcome_col] = (data[intervention_outcome_col] - int_mean) / int_std
    else:
        model_data[intervention_outcome_col] = data[intervention_outcome_col] - int_mean
    feature_transforms[intervention_outcome_col] = {
        "mean": int_mean,
        "std": int_std if int_std > 0 else 1.0,
    }
    
    # Standardize target variable
    target_mean = data[target_col].mean()
    target_std = data[target_col].std()
    if target_std > 0:
        model_data[target_col] = (data[target_col] - target_mean) / target_std
    else:
        model_data[target_col] = data[target_col] - target_mean
    feature_transforms[target_col] = {
        "mean": target_mean,
        "std": target_std if target_std > 0 else 1.0,
    }
    
    # Prepare input features
    X_features = model_data[features].values
    X_intervention = model_data[intervention_outcome_col].values.reshape(-1, 1)
    X_combined = np.concatenate([X_features, X_intervention], axis=1)
    y = model_data[target_col].values
    
    n_samples, n_input = X_combined.shape
    
    with pm.Model() as model:
        X_tensor = pt.as_tensor_variable(X_combined)
        layer_sizes = [n_input] + hidden_units + [1]
        current_input = X_tensor
        
        # Conservative priors for limited data
        weight_sigma = 0.5 / np.sqrt(n_input)
        bias_sigma = 0.1
        
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            W = pm.Normal(f"W_{i}", mu=0, sigma=weight_sigma, shape=(in_size, out_size))
            b = pm.Normal(f"b_{i}", mu=0, sigma=bias_sigma, shape=out_size)
            
            linear_out = pt.dot(current_input, W) + b
            
            if i < len(layer_sizes) - 2:  # Hidden layers
                current_input = pt.maximum(linear_out, 0)  # ReLU activation
            else:  # Output layer
                network_output = linear_out.flatten()
        
        # Noise parameter
        sigma = pm.HalfNormal("sigma", sigma=0.5)
        
        # Likelihood
        y_obs = pm.Normal("y_obs", mu=network_output, sigma=sigma, observed=y)
        
        # Sample with custom parameters
        trace = pm.sample(
            draws=draws, tune=tune, target_accept=0.9, 
            random_seed=42, return_inferencedata=True, 
            progressbar=False, verbose=False, chains=2
        )
    
    return trace, feature_transforms

def predict_bnn_fast_loo(trace, new_data, feature_transforms, features, 
                        intervention_outcome_col, hidden_units=[8], n_samples=1000):
    """
    Fast BNN prediction for leave-one-out evaluation
    """
    # Apply standardization
    standardized_data = new_data.copy()
    for feature in features + [intervention_outcome_col]:
        if feature in standardized_data and feature in feature_transforms:
            transform = feature_transforms[feature]
            mean_val = transform["mean"]
            std_val = transform["std"]
            standardized_data[feature] = (standardized_data[feature] - mean_val) / std_val
    
    # Prepare input vector
    X_features = np.array([standardized_data[f] for f in features])
    X_intervention = np.array([standardized_data[intervention_outcome_col]])
    X_combined = np.concatenate([X_features, X_intervention])
    
    posterior = trace.posterior
    n_chains = posterior.dims["chain"]
    n_draws = posterior.dims["draw"]
    predictions = []
    
    layer_sizes = [len(features) + 1] + hidden_units + [1]
    
    for _ in range(n_samples):
        chain_idx = np.random.randint(0, n_chains)
        draw_idx = np.random.randint(0, n_draws)
        
        current_input = X_combined
        
        for layer_idx, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            W = posterior[f"W_{layer_idx}"][chain_idx, draw_idx].values
            b = posterior[f"b_{layer_idx}"][chain_idx, draw_idx].values
            
            linear_out = np.dot(current_input, W) + b
            
            if layer_idx < len(layer_sizes) - 2:
                current_input = np.maximum(linear_out, 0)  # ReLU
            else:
                network_output = linear_out[0]
        
        sigma_val = posterior["sigma"][chain_idx, draw_idx].values
        pred_sample = np.random.normal(network_output, sigma_val)
        predictions.append(pred_sample)
    
    predictions = np.array(predictions)
    
    # Transform back to original scale
    target_transform = feature_transforms[target_col]
    target_mean = target_transform["mean"]
    target_std = target_transform["std"]
    predictions = predictions * target_std + target_mean
    
    return predictions




In [ ]:
results_bayesian_bnn = []
bayesian_draws = 500
bayesian_tune = 250

for index, row in training_df.iterrows():
    rct_name = row[trial_id_col]
    is_arm_control = row[control_arm_col]
    
    if is_arm_control == 1:
        print(f"\nProcessing {rct_name}...")
        
        # Ground truth
        outcome_control = round(row[target_col], 2)
        
        # ==========================================
        # COLLECT ACTUAL CI DATA FOR IoU CALCULATION
        # ==========================================
        
        # Get the actual CI for the current control arm (for IoU calculation)
        actual_outcome_ci = None
        if "PFS_median_CI" in training_df.columns:
            actual_ci_value = row["PFS_median_CI"]
            if pd.notna(actual_ci_value) and actual_ci_value is not None:
                # Handle different formats of CI storage
                if isinstance(actual_ci_value, (list, tuple, np.ndarray)):
                    actual_outcome_ci = actual_ci_value
                elif isinstance(actual_ci_value, str):
                    try:
                        # Try to parse string representation like "[x, y]"
                        actual_outcome_ci = eval(actual_ci_value)
                    except:
                        print(f"Warning: Could not parse CI string for {rct_name}: {actual_ci_value}")
                        actual_outcome_ci = None
                else:
                    actual_outcome_ci = None
        
        # Get the trt outcome of the RCT targeted
        trt_arm = training_df.loc[training_df[trial_id_col] == rct_name, :]
        trt_outcome = trt_arm.loc[trt_arm[control_arm_col] != 1, target_col]
        if trt_outcome.empty or pd.isna(trt_outcome.mean()):
            continue
        trt_outcome = round(trt_outcome.mean(), 2)
        
        # Prepare training data
        training_j = training_df.copy()
        # Flag inference row
        training_j["is_to_predict"] = np.where(training_j.index == index, 1, 0)
        # Flag RCT row
        training_j["is_targeted_rct"] = np.where(training_j[trial_id_col] == rct_name, 1, 0)
        
        # Add intervention outcome as a feature for all control arms
        for idx, control_row in training_j[training_j[control_arm_col] == 1].iterrows():
            ctrl_rct = control_row[trial_id_col]
            # Get intervention outcome for this control's trial
            ctrl_trt_outcome = training_j.loc[(training_j[trial_id_col] == ctrl_rct) & 
                                             (training_j[control_arm_col] != 1), target_col]
            
            if not ctrl_trt_outcome.empty and not pd.isna(ctrl_trt_outcome.mean()):
                training_j.loc[idx, intervention_outcome_col] = round(ctrl_trt_outcome.mean(), 2)
            else:
                training_j.loc[idx, intervention_outcome_col] = np.nan
        
        training_j = training_j.dropna(subset=[intervention_outcome_col])
        
        # Only consider control arms for training
        training_j = training_j.loc[training_j[control_arm_col] == 1, :]
        training_j = prepare_data(training_j, list_ftr_selected + ["is_to_predict", "is_targeted_rct", intervention_outcome_col, target_col])
        training_j = run_fillna(training_j)
        
        # Make sure the target row has the intervention outcome
        training_j.loc[training_j["is_to_predict"] == 1, intervention_outcome_col] = trt_outcome
        
        # Extract inference data
        inference_df = training_j.loc[training_j["is_to_predict"] == 1, :]
        if inference_df.empty:
            print(f"Skipping {rct_name} as no inference data available.")
            continue
            
        # Remove inference row and targeted RCT from training
        training_subset = training_j.loc[training_j["is_to_predict"] != 1, :]
        training_subset = training_subset.loc[training_subset["is_targeted_rct"] != 1, :]
        
        # Check if we have enough training data
        if training_subset.shape[0] < 10:
            print(f"Skipping {rct_name} due to insufficient data for Bayesian inference.")
            continue
        
        try:
            # ==========================================
            # BUILD BNN AND CALCULATE TRAINING ACCURACY METRICS
            # ==========================================
            
            print(f"  Building BNN model...")
            # Build and sample BNN model
            bnn_trace, bnn_transforms = build_bnn_model_fast_loo(
                training_subset, list_ftr_selected, target_col, intervention_outcome_col,
                hidden_units=[8], draws=bayesian_draws, tune=bayesian_tune
            )
            
            # Get BNN training predictions for evaluation
            bnn_training_preds = []
            for _, train_row in training_subset.iterrows():
                train_data = train_row[list_ftr_selected + [intervention_outcome_col]].to_dict()
                train_pred_samples = predict_bnn_fast_loo(
                    bnn_trace, train_data, bnn_transforms, list_ftr_selected, 
                    intervention_outcome_col, hidden_units=[8], n_samples=500
                )
                bnn_training_preds.append(np.mean(train_pred_samples))
            
            bnn_training_preds = np.array(bnn_training_preds)
            training_true = training_subset[target_col].values
            
            # Calculate BNN training metrics
            bnn_training_rmse = round(np.sqrt(mean_squared_error(training_true, bnn_training_preds)), 3)
            bnn_training_r2 = round(r2_score(training_true, bnn_training_preds), 3)
            bnn_training_mae = round(np.mean(np.abs(training_true - bnn_training_preds)), 3)
            
            print(f"  BNN Training metrics - RMSE: {bnn_training_rmse}, R²: {bnn_training_r2}, MAE: {bnn_training_mae}")
            
            # ==========================================
            # GET BNN TEST PREDICTION WITH UNCERTAINTY
            # ==========================================
            
            # Create prediction data
            new_data = inference_df[list_ftr_selected + [intervention_outcome_col]].iloc[0].to_dict()
            
            # Make BNN predictions with full posterior distribution
            bnn_pred_samples = predict_bnn_fast_loo(
                bnn_trace, new_data, bnn_transforms, list_ftr_selected, 
                intervention_outcome_col, hidden_units=[8], n_samples=2000
            )
            
            # Calculate BNN prediction statistics
            bnn_predicted_outcome = round(np.nanmean(bnn_pred_samples), 2)
            bnn_pred_std = round(np.nanstd(bnn_pred_samples), 2)
            bnn_pred_median = round(np.nanmedian(bnn_pred_samples), 2)
            
            # Calculate BNN confidence intervals
            bnn_pred_ci_95 = [
                round(np.percentile(bnn_pred_samples, 2.5), 2),
                round(np.percentile(bnn_pred_samples, 97.5), 2)
            ]
            bnn_pred_ci_90 = [
                round(np.percentile(bnn_pred_samples, 5.0), 2),
                round(np.percentile(bnn_pred_samples, 95.0), 2)
            ]
            bnn_pred_ci_80 = [
                round(np.percentile(bnn_pred_samples, 10.0), 2),
                round(np.percentile(bnn_pred_samples, 90.0), 2)
            ]
            
            # Additional BNN uncertainty measures
            bnn_pred_ci_width = round(bnn_pred_ci_95[1] - bnn_pred_ci_95[0], 3)
            bnn_pred_iqr = round(np.percentile(bnn_pred_samples, 75) - np.percentile(bnn_pred_samples, 25), 3)
            
            # ==========================================
            # CALCULATE BNN ATE DISTRIBUTION
            # ==========================================
            
            # Calculate BNN ATEs
            real_ate = round(trt_outcome - outcome_control, 2)
            bnn_pred_ate = round(trt_outcome - bnn_predicted_outcome, 2)
            
            # Calculate BNN ATE distribution (treatment outcome - predicted control distribution)
            bnn_ate_samples = trt_outcome - bnn_pred_samples
            bnn_ate_mean = round(np.mean(bnn_ate_samples), 2)
            bnn_ate_std = round(np.std(bnn_ate_samples), 2)
            bnn_ate_ci_95 = [
                round(np.percentile(bnn_ate_samples, 2.5), 2),
                round(np.percentile(bnn_ate_samples, 97.5), 2)
            ]
            bnn_ate_ci_90 = [
                round(np.percentile(bnn_ate_samples, 5.0), 2),
                round(np.percentile(bnn_ate_samples, 95.0), 2)
            ]
            bnn_ate_ci_width = round(bnn_ate_ci_95[1] - bnn_ate_ci_95[0], 3)
            
            # BNN Probability of positive ATE
            bnn_prob_positive_ate = round(np.mean(bnn_ate_samples > 0), 3)
            
            # BNN Empirical-based ATE metrics (alternative calculation)
            bnn_ate_ci_95_empirical = [
                round(trt_outcome - bnn_pred_ci_95[1], 2),  # Note: reversed order
                round(trt_outcome - bnn_pred_ci_95[0], 2)
            ]
            bnn_ate_ci_90_empirical = [
                round(trt_outcome - bnn_pred_ci_90[1], 2),
                round(trt_outcome - bnn_pred_ci_90[0], 2)
            ]
            
            # Alternative BNN probability calculation using empirical quantiles
            if bnn_pred_ci_95[1] < trt_outcome:
                bnn_prob_positive_ate_empirical = 1.0
            elif bnn_pred_ci_95[0] > trt_outcome:
                bnn_prob_positive_ate_empirical = 0.0
            else:
                bnn_prob_positive_ate_empirical = bnn_prob_positive_ate
            
            # ==========================================
            # STORE BNN RESULTS WITH ACTUAL CI
            # ==========================================
            
            results_bayesian_bnn.append({
                # Original results
                "real_ate": real_ate,
                "pred_ate": bnn_pred_ate,
                "outcome_control": outcome_control,
                "predicted_outcome": bnn_predicted_outcome,
                "rct_name": rct_name,
                "intervention": row["intervention"],
                "Arm": row["Arm"],
                "n_training_samples": len(training_subset),
                
                # BNN ATE distribution metrics (needed for performance evaluation)
                "ate_mean": bnn_ate_mean,
                "ate_std": bnn_ate_std,
                "ate_ci_95": bnn_ate_ci_95,
                "ate_ci_90": bnn_ate_ci_90,
                "ate_ci_width": bnn_ate_ci_width,
                "ate_samples": bnn_ate_samples.tolist(),
                "prob_positive_ate": bnn_prob_positive_ate,
                
                # Enhanced BNN prediction uncertainty metrics
                "pred_std": bnn_pred_std,
                "pred_median": bnn_pred_median,
                "pred_ci_95": bnn_pred_ci_95,
                "pred_ci_90": bnn_pred_ci_90,
                "pred_ci_80": bnn_pred_ci_80,
                "pred_ci_width": bnn_pred_ci_width,
                "pred_iqr": bnn_pred_iqr,
                "pred_samples": bnn_pred_samples.tolist(),
                
                # Empirical-based BNN ATE metrics (more accurate)
                "ate_ci_95_empirical": bnn_ate_ci_95_empirical,
                "ate_ci_90_empirical": bnn_ate_ci_90_empirical,
                "prob_positive_ate_empirical": bnn_prob_positive_ate_empirical,
                
                # BNN Training accuracy metrics
                "training_rmse": bnn_training_rmse,
                "training_r2": bnn_training_r2,
                "training_mae": bnn_training_mae,
                
                # Additional training set statistics
                "training_set_std": round(np.std(training_true), 3),
                "training_pred_std": round(np.std(bnn_training_preds), 3),
                
                # NEW: Data for IoU calculation
                "actual_outcome_ci": actual_outcome_ci,  # Actual CI from RCT data
                "original_row_index": index,  # Store original index for potential merging
                
                # Additional BNN uncertainty measures for comprehensive evaluation
                "pred_range_90": round(bnn_pred_ci_90[1] - bnn_pred_ci_90[0], 3),
                "pred_range_80": round(bnn_pred_ci_80[1] - bnn_pred_ci_80[0], 3),
            })
            
            print(f"{rct_name} - arm: {row['Arm']} - intervention: {trt_outcome}")
            print(f"  BNN Real outcome: {outcome_control}, Pred: {bnn_predicted_outcome} ± {bnn_pred_std}")
            print(f"  BNN Pred CI 95%: {bnn_pred_ci_95}, width: {bnn_pred_ci_width}")
            if actual_outcome_ci:
                print(f"  Actual CI: {actual_outcome_ci}")
            print(f"  BNN Real ATE: {real_ate}, Pred ATE: {bnn_pred_ate} (95% CI: {bnn_ate_ci_95})")
            print(f"  BNN P(ATE > 0): {bnn_prob_positive_ate} (empirical: {bnn_prob_positive_ate_empirical})")
            
        except Exception as e:
            print(f"BNN Error with {rct_name}: {str(e)}")
            continue

In [ ]:
# Convert to DataFrame
results_bayesian_bnn_df = pd.DataFrame(results_bayesian_bnn)
print(f"BNN Results DataFrame shape: {results_bayesian_bnn_df.shape}")

# Display the results
results_bayesian_bnn_df.head(2)

# Gaussian Process
- Custom mean function = multiplier * intervention_outcome. 
- Multiplier is estimated from Dynamic Ratio Estimation method(from prev. notebook)
- Matern32 Kernel

Choice of kernel depends on data

### More Features

In [ ]:
%autoreload 2

In [ ]:
from utilities.gp_utils import build_gp_with_informative_prior, predict_gp_with_uncertainty, compute_matern32_kernel
from utilities.utils import select_features_with_correlation_analysis, calculate_contextual_prior

In [ ]:
training_df['age_risk'] = (training_df['age_median'] > 65).astype(float)
training_df['egfr_treatment_match'] = training_df['EGFR_positive_mutation'] * training_df['EGFR_TKI']
training_df['immunotherapy_eligible'] = (1 - training_df['brain_metastase_yes']) * training_df['PD1_PDL1_Inhibitor']
training_df['vegf_chemo_combo'] = training_df['Anti_VEGF'] * training_df['Chemotherapy']
training_df['immuno_targeted_combo'] = training_df['Immunotherapy'] * training_df['Targeted_Therapy']
training_df['platinum_taxane_combo'] = training_df['Platinum_Chemotherapy'] * training_df['Taxane']


In [ ]:
# Combine all features
list_ftr_selected = ['age_median',
 'gender_male_percent',
 'ecog_1',
 'EGFR_positive_mutation',
 'disease_stage_IV',
 'disease_stage_recurrent',
 'PD1_PDL1_Inhibitor',
 'EGFR_TKI',
 'Platinum_Chemotherapy',
 'Anti_VEGF',
 'EGFR_wild',
 'disease_stage_III',
 'egfr_targeted',
 'egfr_tki_use',
 'Chemotherapy',
 'Immunotherapy',
 'vegf_chemo_combo',
 'platinum_taxane_combo'
 ] \
+ ["age_risk", "egfr_treatment_match", "immunotherapy_eligible"]

In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import spearmanr, pearsonr
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ========================================
# MAIN TRAINING LOOP WITH UNCERTAINTY
# ========================================

print("Starting GP with Informative Priors and Full Uncertainty...")
print("="*70)

results_gp_uncertainty = []

for index, row in training_df.iterrows():
    rct_name = row[trial_id_col]
    is_arm_control = row[control_arm_col]
    
    if is_arm_control == 1:
        print(f"\nProcessing {rct_name}...")
        
        # Ground truth
        outcome_control = round(row[target_col], 2)

        # Get actual CI if available
        actual_outcome_ci = None
        if "PFS_median_CI" in training_df.columns:
            actual_ci_value = row["PFS_median_CI"]
            if pd.notna(actual_ci_value) and actual_ci_value is not None:
                if isinstance(actual_ci_value, (list, tuple, np.ndarray)):
                    actual_outcome_ci = actual_ci_value
                elif isinstance(actual_ci_value, str):
                    try:
                        actual_outcome_ci = eval(actual_ci_value)
                    except:
                        actual_outcome_ci = None
        
        # Get treatment outcome
        trt_arm = training_df.loc[training_df[trial_id_col] == rct_name, :]
        trt_outcome = trt_arm.loc[trt_arm[control_arm_col] != 1, target_col]
        if trt_outcome.empty or pd.isna(trt_outcome.mean()):
            continue
        trt_outcome = round(trt_outcome.mean(), 2)
        
        # Prepare training data (same as before)
        training_j = training_df.copy()
        training_j["is_to_predict"] = np.where(training_j.index == index, 1, 0)
        training_j["is_targeted_rct"] = np.where(training_j[trial_id_col] == rct_name, 1, 0)
        
        # Add intervention outcomes
        for idx, control_row in training_j[training_j[control_arm_col] == 1].iterrows():
            ctrl_rct = control_row[trial_id_col]
            ctrl_trt_outcome = training_j.loc[(training_j[trial_id_col] == ctrl_rct) & 
                                             (training_j[control_arm_col] != 1), target_col]
            
            if not ctrl_trt_outcome.empty and not pd.isna(ctrl_trt_outcome.mean()):
                training_j.loc[idx, intervention_outcome_col] = round(ctrl_trt_outcome.mean(), 2)
            else:
                training_j.loc[idx, intervention_outcome_col] = np.nan
        
        training_j = training_j.dropna(subset=[intervention_outcome_col])
        training_j = training_j.loc[training_j[control_arm_col] == 1, :]
        
        # Prepare features
        training_j = prepare_data(training_j, list_ftr_selected + ["is_to_predict", "is_targeted_rct", intervention_outcome_col, target_col])
        training_j = run_fillna(training_j)
        
        # Set target intervention outcome
        training_j.loc[training_j["is_to_predict"] == 1, intervention_outcome_col] = trt_outcome
        
        # Extract inference and training data
        inference_df = training_j.loc[training_j["is_to_predict"] == 1, :]
        if inference_df.empty:
            continue
            
        training_subset = training_j.loc[training_j["is_to_predict"] != 1, :]
        training_subset = training_subset.loc[training_subset["is_targeted_rct"] != 1, :]
        
        if training_subset.shape[0] < 12:
            continue
        
        try:
            # ==========================================
            # FEATURE SELECTION
            # ==========================================
            max_features = min(5, max(3, len(training_subset) // 5))
            selected_features = select_features_with_correlation_analysis(
                training_subset, list_ftr_selected, target_col, max_features
            )
            print(f"  Selected {len(selected_features)} features for {len(training_subset)} samples")
            
            # ==========================================
            # BUILD GP WITH INFORMATIVE PRIOR
            # ==========================================
            trace, gp_params = build_gp_with_informative_prior(
                training_subset, selected_features, target_col, 
                intervention_outcome_col, trt_outcome,
                draws=400, tune=400
            )
            
            # Check convergence
            divergences = trace.sample_stats.diverging.sum().values
            if divergences > 20:
                print(f"  Warning: {divergences} divergences, using fallback")
                # Use simple baseline with uncertainty
                baseline_pred = 0.712 * trt_outcome
                baseline_std = training_subset[target_col].std()
                pred_samples = np.random.normal(baseline_pred, baseline_std, size=1000)
                pred_samples = np.clip(pred_samples, 0, None)
            else:
                print(f"  GP converged with {divergences} divergences")
                
                # ==========================================
                # POSTERIOR PREDICTIVE SAMPLING
                # ==========================================
                inference_data = inference_df.iloc[0].to_dict()
                pred_samples = predict_gp_with_uncertainty(
                    trace, gp_params, inference_data, n_samples=1500
                )
            
            # ==========================================
            # UNCERTAINTY QUANTIFICATION
            # ==========================================
            if len(pred_samples) > 10:
                pred_mean = np.mean(pred_samples)
                pred_median = np.median(pred_samples)
                pred_std = np.std(pred_samples)
                
                # Confidence intervals
                ci_95 = [np.percentile(pred_samples, 2.5), np.percentile(pred_samples, 97.5)]
                ci_90 = [np.percentile(pred_samples, 5.0), np.percentile(pred_samples, 95.0)]
                ci_68 = [np.percentile(pred_samples, 16.0), np.percentile(pred_samples, 84.0)]
                
                # Prediction intervals
                pred_mean = round(pred_mean, 2)
                pred_std = round(pred_std, 2)
                
                # ATE posterior
                ate_samples = trt_outcome - pred_samples
                ate_mean = np.mean(ate_samples)
                ate_std = np.std(ate_samples)
                ate_ci_95 = [np.percentile(ate_samples, 2.5), np.percentile(ate_samples, 97.5)]
                
                # Probability of positive ATE
                prob_positive_ate = np.mean(ate_samples > 0)
                
            else:
                # Fallback
                pred_mean = 0.712 * trt_outcome # 0.712 came from dynamic ratio estimate
                pred_std = training_subset[target_col].std()
                ci_95 = [pred_mean - 1.96*pred_std, pred_mean + 1.96*pred_std]
                ci_90 = [pred_mean - 1.64*pred_std, pred_mean + 1.64*pred_std]
                ci_68 = [pred_mean - pred_std, pred_mean + pred_std]
                ate_mean = trt_outcome - pred_mean
                ate_std = pred_std
                ate_ci_95 = [ate_mean - 1.96*ate_std, ate_mean + 1.96*ate_std]
                prob_positive_ate = 0.5
            
            # ==========================================
            # RESULTS WITH FULL UNCERTAINTY
            # ==========================================
            real_ate = round(trt_outcome - outcome_control, 2)
            
            results_gp_uncertainty.append({
                "rct_name": rct_name,
                "real_ate": real_ate,
                "pred_ate": round(trt_outcome - pred_mean, 2),
                "ate_mean": round(ate_mean, 2),
                "outcome_control": outcome_control,
                "predicted_mean": pred_mean,
                "predicted_outcome": pred_mean,
                "predicted_std": pred_std,
                "predicted_median": round(pred_median, 2),
                "ci_95": [round(ci_95[0], 2), round(ci_95[1], 2)],
                "pred_ci_95": [round(ci_95[0], 2), round(ci_95[1], 2)],
                "ci_90": [round(ci_90[0], 2), round(ci_90[1], 2)],
                "ci_68": [round(ci_68[0], 2), round(ci_68[1], 2)],
                "ate_std": round(ate_std, 2),
                "ate_ci_95": [round(ate_ci_95[0], 2), round(ate_ci_95[1], 2)],
                "prob_positive_ate": round(prob_positive_ate, 3),
                "intervention": row.get("intervention", ""),
                "n_training_samples": len(training_subset),
                "n_features": len(selected_features),
                "divergences": int(divergences),
                "actual_outcome_ci": actual_outcome_ci,
                "prior_mean": round(gp_params.get('prior_mean', 0.712 * trt_outcome), 2)
            })
            
            print(f"  Prediction: {pred_mean:.2f} ± {pred_std:.2f}")
            print(f"  95% CI: [{ci_95[0]:.2f}, {ci_95[1]:.2f}]")
            print(f"  ATE: {ate_mean:.2f} ± {ate_std:.2f}")
            print(f"  P(ATE > 0): {prob_positive_ate:.3f}")
            print(f"  Actual: {outcome_control}")
            
        except Exception as e:
            print(f"  Error: {str(e)}")
            continue



In [ ]:
# ==========================================
# ANALYSIS WITH UNCERTAINTY METRICS
# ==========================================

if results_gp_uncertainty:
    results_df = pd.DataFrame(results_gp_uncertainty)
    
    # Point prediction metrics
    rmse = np.sqrt(mean_squared_error(results_df['outcome_control'], results_df['predicted_mean']))
    mae = mean_absolute_error(results_df['outcome_control'], results_df['predicted_mean'])
    r2 = r2_score(results_df['outcome_control'], results_df['predicted_mean'])
    
    # Uncertainty calibration
    # Check if true values fall within confidence intervals
    ci_95_coverage = 0
    ci_90_coverage = 0
    ci_68_coverage = 0
    
    for _, row in results_df.iterrows():
        true_val = row['outcome_control']
        if row['ci_95'][0] <= true_val <= row['ci_95'][1]:
            ci_95_coverage += 1
        if row['ci_90'][0] <= true_val <= row['ci_90'][1]:
            ci_90_coverage += 1
        if row['ci_68'][0] <= true_val <= row['ci_68'][1]:
            ci_68_coverage += 1
    
    ci_95_coverage_pct = ci_95_coverage / len(results_df) * 100
    ci_90_coverage_pct = ci_90_coverage / len(results_df) * 100
    ci_68_coverage_pct = ci_68_coverage / len(results_df) * 100
    
    print("\n" + "="*70)
    print("GP WITH UNCERTAINTY QUANTIFICATION RESULTS")
    print("="*70)
    print(f"Number of predictions: {len(results_df)}")
    
    print(f"\nPoint Prediction Performance:")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  MAE: {mae:.3f}")
    print(f"  R²: {r2:.3f}")
    
    print(f"\nUncertainty Calibration:")
    print(f"  95% CI coverage: {ci_95_coverage_pct:.1f}% (target: 95%)")
    print(f"  90% CI coverage: {ci_90_coverage_pct:.1f}% (target: 90%)")
    print(f"  68% CI coverage: {ci_68_coverage_pct:.1f}% (target: 68%)")
    
    print(f"\nUncertainty Statistics:")
    print(f"  Average prediction std: {results_df['predicted_std'].mean():.2f}")
    print(f"  Average 95% CI width: {(results_df['ci_95'].apply(lambda x: x[1] - x[0])).mean():.2f}")
    

else:
    print("\nNo results generated")

In [ ]:
results_bayesian_gp_df = pd.DataFrame.from_dict(results_gp_uncertainty)
results_bayesian_gp_df['predicted_outcome'] = results_bayesian_gp_df['predicted_mean']
results_bayesian_gp_df['pred_ci_95'] = results_bayesian_gp_df['ci_95']
results_bayesian_gp_df.head(2)

# Results

![Results](../images/rct_lean_results_demo.png)

In [ ]:
from utilities.utils import calculate_perf_enhanced_with_iou

In [ ]:
pd.DataFrame([
calculate_perf_enhanced_with_iou(results_bayesian_df.dropna(subset=['real_ate', 'pred_ate']), "Bayesian LR Approach",
                                  actual_ci_col='actual_outcome_ci',
                                  pred_ci_col='pred_ci_95'),
calculate_perf_enhanced_with_iou(results_bayesian_bnn_df.dropna(subset=['real_ate', 'pred_ate']), "Bayesian BNN Approach",
                                  actual_ci_col='actual_outcome_ci',
                                  pred_ci_col='pred_ci_95'),
calculate_perf_enhanced_with_iou(results_bayesian_gp_df.dropna(subset=['real_ate', 'pred_ate']), "Bayesian GP Approach",
                                  actual_ci_col='actual_outcome_ci',
                                  pred_ci_col='pred_ci_95'),
])[['Approach',
 'ATE direction true',
 'r2_ate',
 'spearman_ate',
 'rmse_ate',
 'r2_outcome',
 'rmse_outcome',
 'abs_bias_ate',
 'avg_ci_width_ate',
 'avg_iou_outcome',
 'median_iou_outcome'
]]

In [ ]:
iou_outcomes_scores = pd.DataFrame([calculate_perf_enhanced_with_iou(results_bayesian_df.dropna(), "Bayesian LR Approach",
                                  actual_ci_col='actual_outcome_ci',
                                  pred_ci_col='pred_ci_95')])
iou_outcomes_scores['iou_outcome_scores'].tolist()[0]

In [ ]:
iou_results_bayesian_df = results_bayesian_df.dropna().dropna().copy()
iou_results_bayesian_df['iou_outcome_scores'] = iou_outcomes_scores['iou_outcome_scores'].tolist()[0]
iou_results_bayesian_df[['real_ate', 'pred_ate', 'outcome_control', 'predicted_outcome',
                         'rct_name', 'intervention', 'Arm', 'actual_outcome_ci', 'pred_ci_95', 'iou_outcome_scores']]

# Conclusion

Tree based models still outperforms Bayesian linear model approach with informed prior (**1.08** vs **1.31**). It performs slightly worse than 70% ratio method (**1.20**) (one of our baseline approaches).
